- Checking for api key being loaded or not

In [18]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")

def fetch_timetable(board_type, stop_id):
    """
    Function to fetch timetable data from the ResRobot API.
    
    Args:
        board_type (str): 'departureBoard' or 'arrivalBoard'
        stop_id (str): The stop ID for Göteborg Centralstationen
    
    Returns:
        dict: Parsed JSON response from the API, or None if an error occurs
    """
    base_url = f"https://api.resrobot.se/v2.1/{board_type}"
    params = {
        "id": stop_id,
        "format": "json",
        "accessId": API_KEY
    }
    
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def count_transports(data):
    """
    Function to count the number of transports in the timetable data.
    
    Args:
        data (dict): JSON response from the API
    
    Returns:
        int: Number of transport entries
    """
    if data and "Departure" in data:
        return len(data["Departure"])
    elif data and "Arrival" in data:
        return len(data["Arrival"])
    else:
        return 0


stop_id = "740000002"


arrival_data = fetch_timetable("arrivalBoard", stop_id)
arrival_count = count_transports(arrival_data)
print(f"Number of transports arriving at Göteborg Centralstationen: {arrival_count}")

departure_data = fetch_timetable("departureBoard", stop_id)
departure_count = count_transports(departure_data)
print(f"Number of transports departing from Göteborg Centralstationen: {departure_count}")


Number of transports arriving at Göteborg Centralstationen: 120
Number of transports departing from Göteborg Centralstationen: 116


In [ ]:
import requests
import os
from dotenv import load_dotenv
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
import pandas as pd
from time import sleep


load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")


def fetch_timetable(board_type, stop_id):
    """Fetch timetable data from ResRobot API."""
    base_url = f"https://api.resrobot.se/v2.1/{board_type}"
    params = {
        "id": stop_id,
        "format": "json",
        "accessId": API_KEY
    }
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract_trams(data):
    """Extract trams from the timetable data."""
    trams = []
    if data and "Departure" in data:
        for departure in data["Departure"]:
            if "spårväg" in departure.get("ProductAtStop", {}).get("name", "").lower():
                trams.append({
                    "Destination": departure["direction"],
                    "Time": departure["date"] + " " + departure["time"]
                })
    return trams

def clean_destination(destination):
    """Clean destination names by removing parentheses and extra identifiers."""
    return destination.split(" (")[0]


def geocode_with_retry(destination, geolocator, retries=3, delay=1):
    """Retry geocoding a destination with a delay between attempts."""
    for attempt in range(retries):
        try:
            return geolocator.geocode(destination)
        except GeocoderTimedOut:
            print(f"Retrying ({attempt + 1}/{retries}) for: {destination}")
            sleep(delay)
    print(f"Failed to geocode after retries: {destination}")
    return None


def geocode_locations(trams):
    """Get coordinates for tram destinations using refined geocoding."""
    geolocator = Nominatim(user_agent="tram_geocoder")
    coordinates = []
    failed = []
    for tram in trams:
        destination_cleaned = clean_destination(tram["Destination"])
        location = geocode_with_retry(destination_cleaned, geolocator)
        if location:
            coordinates.append({
                "Destination": tram["Destination"],
                "Time": tram["Time"],
                "Latitude": location.latitude,
                "Longitude": location.longitude
            })
        else:
            failed.append(tram["Destination"])

    pd.DataFrame({"Failed Destination": failed}).to_csv("failed_geocodes.csv", index=False)
    print(f"Failed destinations saved to failed_geocodes.csv")
    return coordinates


def plot_tram_locations(coordinates):
    """Plot tram destinations on a map."""
    tram_map = folium.Map(location=[57.7089, 11.9746], zoom_start=12)
    if not coordinates:
    
        folium.Marker(
            location=[57.7089, 11.9746],
            popup="No valid tram data available",
            icon=folium.Icon(color="red")
        ).add_to(tram_map)
    else:
        for coord in coordinates:
            folium.Marker(
                location=[coord["Latitude"], coord["Longitude"]],
                popup=f"{coord['Destination']}<br>{coord['Time']}",
                icon=folium.Icon(color="blue", icon="info-sign")
            ).add_to(tram_map)
    return tram_map


def save_tram_data(trams):
    """Save tram data to a CSV file."""
    df = pd.DataFrame(trams)
    df.to_csv("tram_data.csv", index=False)
    print("Tram data saved to tram_data.csv")


def main():

    stop_id = "740000002"
    

    departure_data = fetch_timetable("departureBoard", stop_id)
    
 
    trams = extract_trams(departure_data)
    

    save_tram_data(trams)
    
    tram_coordinates = geocode_locations(trams)

    tram_map = plot_tram_locations(tram_coordinates)
    tram_map.save("tram_map.html")
    print("Map saved as tram_map.html")

if __name__ == "__main__":
    main()


Tram data saved to tram_data.csv
Failed destinations saved to failed_geocodes.csv
Map saved as tram_map.html


In [28]:
import requests
import os
from dotenv import load_dotenv
import folium


load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")

def fetch_stops(city):
    """Fetch stop details for a given city using the location.name endpoint."""
    base_url = "https://api.resrobot.se/v2.1/location.name"
    params = {
        "input": city,
        "format": "json",
        "accessId": API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f"Raw Response for {city}: {data}")  
        stop_data = []
        if "stopLocationOrCoordLocation" in data:
            for item in data["stopLocationOrCoordLocation"]:
                if "StopLocation" in item:
                    stop = item["StopLocation"]
                    stop_data.append({
                        "Name": stop.get("name", "Unknown"),
                        "Latitude": stop.get("lat"),
                        "Longitude": stop.get("lon"),
                        "ID": stop.get("id")
                    })
            return stop_data
        else:
            print(f"No valid stop data found for {city}.")
    else:
        print(f"Error: {response.status_code}, {response.text}")
    return []

def plot_stops_on_map(coordinates, city_a, city_b):
    """Plot stops on a map."""
    if not coordinates:
        print(f"No coordinates available for route between {city_a} and {city_b}.")
        return
    map_center = [coordinates[0]["Latitude"], coordinates[0]["Longitude"]]
    route_map = folium.Map(location=map_center, zoom_start=10)
    for coord in coordinates:
        folium.Marker(
            location=[coord["Latitude"], coord["Longitude"]],
            popup=coord["Name"],
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(route_map)
    route_map.save(f"{city_a}_to_{city_b}_route_map.html")
    print(f"Map saved as {city_a}_to_{city_b}_route_map.html")

def main():
    city_a = "Göteborg"
    city_b = "Lund"
    
    print(f"Fetching stops for {city_a}...")
    stops_a = fetch_stops(city_a)
    print(f"Fetching stops for {city_b}...")
    stops_b = fetch_stops(city_b)
    
    
    all_stops = stops_a + stops_b
    if not all_stops:
        print("No stops found for the specified cities. Exiting...")
        return
    
    plot_stops_on_map(all_stops, city_a, city_b)

if __name__ == "__main__":
    main()


Fetching stops for Göteborg...
Raw Response for Göteborg: {'stopLocationOrCoordLocation': [{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'}, {'icon': {'res': 'prod_gen'}, 'cls': '4'}, {'icon': {'res': 'prod_gen'}, 'cls': '8'}, {'icon': {'res': 'prod_gen'}, 'cls': '16'}, {'icon': {'res': 'prod_gen'}, 'cls': '64'}, {'icon': {'res': 'prod_gen'}, 'cls': '128'}, {'icon': {'res': 'prod_gen'}, 'cls': '256'}], 'timezoneOffset': 60, 'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1737520614@', 'extId': '740098001', 'name': 'GÖTEBORG', 'lon': 11.973479, 'lat': 57.708895, 'weight': 15371, 'products': 222, 'meta': True, 'minimumChangeDuration': 'PT26M'}}, {'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'}, {'icon': {'res': 'prod_gen'}, 'cls': '4'}, {'icon': {'res': 'prod_gen'}, 'cls': '8'}, {'icon': {'res': 'prod_gen'}, 'cls': '16'}, {'icon': {'res': 'prod_gen'}, 'cls': '64'}, {'icon': {'res': 'prod_gen'}, 'cls': '128'}], 't

In [ ]:
import csv
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
import folium


load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")


def fetch_board(board_type, stop_id):
    base_url = f"https://api.resrobot.se/v2.1/{board_type}Board"
    params = {
        "id": stop_id,
        "format": "json",
        "accessId": API_KEY,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json().get(board_type.capitalize(), [])
    else:
        print(f"Failed to fetch {board_type} board. Status Code: {response.status_code}")
        return []


def process_board_data(board, board_type="departure"):
    current_time = datetime.now()
    processed_data = []
    for item in board:
        product = item.get("Product", [{}])
        if isinstance(product, list):
            product = product[0]
        transport_type = product.get("catOutL", "Unknown")
        line = product.get("line", "Unknown")
        destination = item.get("direction", "Unknown")
        time_str = f"{item['date']} {item['time']}"
        departure_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
        minutes_left = (departure_time - current_time).total_seconds() // 60
        processed_data.append({
            "Board Type": board_type,
            "Transport Type": transport_type,
            "Line": line,
            "Destination": destination,
            "Time": departure_time.strftime("%Y-%m-%d %H:%M:%S"),
            "Minutes Left": max(0, int(minutes_left)),
        })
    return processed_data


def save_to_csv(data, filename):
    if not data:
        print(f"No data to save for {filename}")
        return
    keys = data[0].keys()
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"Data saved to {filename}")


def fetch_stops(city):
    base_url = "https://api.resrobot.se/v2.1/location.name"
    params = {
        "input": city,
        "format": "json",
        "accessId": API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        stop_data = []
        if "stopLocationOrCoordLocation" in data:
            for item in data["stopLocationOrCoordLocation"]:
                if "StopLocation" in item:
                    stop = item["StopLocation"]
                    stop_data.append({
                        "Name": stop.get("name", "Unknown"),
                        "Latitude": stop.get("lat"),
                        "Longitude": stop.get("lon"),
                        "ID": stop.get("id")
                    })
            return stop_data
        else:
            print(f"No valid stop data found for {city}.")
    else:
        print(f"Error fetching stops for {city}: {response.status_code}")
    return []


def plot_stops_on_map(stops, city_a, city_b):
    if not stops:
        print(f"No stops available to plot between {city_a} and {city_b}.")
        return
    map_center = [stops[0]["Latitude"], stops[0]["Longitude"]]
    route_map = folium.Map(location=map_center, zoom_start=10)
    for stop in stops:
        folium.Marker(
            location=[stop["Latitude"], stop["Longitude"]],
            popup=stop["Name"],
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(route_map)
    map_filename = f"{city_a}_to_{city_b}_route_map.html"
    route_map.save(map_filename)
    print(f"Map saved as {map_filename}")


def main():
    stop_id = "740015578"  
    city_a = "Göteborg"
    city_b = "Lund"

    
    departures = fetch_board("departure", stop_id)
    arrivals = fetch_board("arrival", stop_id)

    departure_data = process_board_data(departures, board_type="departure")
    arrival_data = process_board_data(arrivals, board_type="arrival")

    save_to_csv(departure_data, "departure_board.csv")
    save_to_csv(arrival_data, "arrival_board.csv")

    
    stops_a = fetch_stops(city_a)
    stops_b = fetch_stops(city_b)

    all_stops = stops_a + stops_b
    save_to_csv(all_stops, "stops_data.csv")

    plot_stops_on_map(all_stops, city_a, city_b)

if __name__ == "__main__":
    main()


Data saved to departure_board.csv
Data saved to arrival_board.csv
Data saved to stops_data.csv
Map saved as Göteborg_to_Lund_route_map.html
